In [3]:
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider, Button, VBox, HBox, Output
import ipywidgets as widgets
from IPython.display import display, clear_output

In [4]:
#Give price of option according to Black-Scholes formula (no dividends)

def black_scholes(call, S , K, t, r, sigma):
    d_one = (np.log(S/K) + (r + (sigma**2)/2)*t)/(sigma*np.sqrt(t))
    
    d_two = d_one - sigma*np.sqrt(t)

    res = S*stats.norm.cdf(d_one) - (stats.norm.cdf(d_two)*K*np.exp(-(r*t)))
    if call:
        return res
    else:
        return res - S + K*np.exp(-(r * t)) 

In [11]:

def update_heatmap(call, S, t, r, price_range, vol_range, detail):
    
    volatility_range = np.linspace(0.01,vol_range, detail)
    strike_range = np.linspace(S-price_range,S+price_range, detail)

    option_prices = np.zeros((len(volatility_range), len(strike_range)))

    for i, sigma in enumerate(volatility_range):
        for j, K in enumerate(strike_range):
            option_prices[i,j] = black_scholes(call, S, K, t, r, sigma)

    fig, ax = plt.subplots(figsize=(10,8))
    cax = ax.imshow(option_prices, extent=[strike_range.min() - (strike_range[1] - strike_range[0]) / 2,
                                           strike_range.max() + (strike_range[1] - strike_range[0]) / 2,
                                           volatility_range.min()- (volatility_range[1] - volatility_range[0]) / 2,
                                           volatility_range.max()+ (volatility_range[1] - volatility_range[0]) / 2],
                   origin="lower", aspect="auto", cmap="plasma")

    if call:
        option_type= "Call"
    else:
        option_type="Put"
        
    title = option_type +  " option price evolution relative to volatility and strike price"
    label = option_type + " option price"
        
    plt.colorbar(cax, label=label)
    plt.title(title)
    plt.xlabel("Strike price (K)")
    plt.ylabel("Volatility (σ)")
    

    ax.set_xticks(strike_range)  # Align x-ticks with the strike prices
    ax.set_yticks(volatility_range)  # Align y-ticks with the volatility values

    # Optionally format the tick labels if there are too many
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{val:.0f}'))
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{val:.2f}'))
    
    fontsize =10
    if detail==20:
        fontsize = 7
    if detail == 25:
        fontsize = 6
    
    for i in range(option_prices.shape[0]):
        for j in range(option_prices.shape[1]):
            x_center = strike_range[j] 
            y_center = volatility_range[i] 
            
            plt.text(x_center, y_center, f'{option_prices[i,j]:.2f}',
                     ha='center', va='center', color='white', fontsize=fontsize)
    
    plt.show()

In [12]:
S = FloatSlider(min=50, max=150, step=1, value=100, description="Spot Price", 
                layout={'width': '500px'})
t = FloatSlider(min=1/365, max=3, step=3/365, value=1, description="Time to Expiry (years)", 
                layout={'width': '500px'})
r = FloatSlider(min=0.00, max=0.1, step=0.005, value=0.04, description="Risk-Free Rate", 
                layout={'width': '500px'})
price_range = IntSlider(min=5, max=50, step=1, value=10, description="K range", 
                        layout={'width': '500px'})
vol_range = FloatSlider(min=0.1, max=3, step=0.01, value=1, description="σ range", 
                        layout={'width': '500px'})
detail = IntSlider(min=5, max=25, step=5, value=10, description="Grid Detail", 
                   layout={'width': '500px'})

button = Button(description="Calculate", button_style='success', 
                layout={'width': '200px', 'padding': '1px'})

output=Output()

def button_click(b):
    with output:
        clear_output(wait="True")
        update_heatmap(True, S.value, t.value, r.value, price_range.value,vol_range.value, detail.value)
        update_heatmap(False, S.value, t.value, r.value, price_range.value, vol_range.value, detail.value)
        

button.on_click(button_click)

ui = VBox([
    HBox([S]), 
    HBox([t]), 
    HBox([r]), 
    HBox([price_range]), 
    HBox([vol_range]), 
    HBox([detail]),
    HBox([button]), 
    output
])

display(ui)